In [1]:
'''Retrieve all inputs executed in the current kernel section'''
%history

'''Retrieve all inputs executed in the current kernel section'''
%history


In [2]:
import numpy as np
import pandas as pd

import warnings

In [3]:
print('Pandas version -> %s' % pd.__version__)
print('Numpy version -> %s' % np.__version__)

Pandas version -> 2.1.2
Numpy version -> 1.26.1


In [4]:
#Read data
df_distance = pd.read_csv('store_location_analysis_distance.csv', index_col='CustomerNeighborhood')
df_stores = pd.read_csv('store_location_analysis_stores.csv', index_col='StoreLocation')
df_census = pd.read_csv('store_location_analysis_census.csv', index_col='CustomerNeighborhood')

In [5]:
df_distance

,Shopping_Mall_01,Shopping_Mall_02,Waterfront_Park,Church_Street
CustomerNeighborhood,,,,
Neighborhood_01,50,70,70,30
Neighborhood_02,70,80,80,40
Neighborhood_03,80,1,70,20
Neighborhood_04,1,20,40,40
Neighborhood_05,10,30,30,30


In [9]:
df_stores.style.format(thousands=',')

,Size,Parking spaces,BusLanes,Accessibility,Location
StoreLocation,,,,,
Shopping_Mall_01,"1,000",400,4,10,6
Shopping_Mall_02,"1,000",300,2,9,6
Waterfront_Park,600,500,3,8,10
Church_Street,800,100,6,8,8


In [7]:
#Total Expenditure by Neighborhood
df_census['Total'] = df_census['Households']*df_census['AvgExpenditure']

df_census.style.format(thousands=',')

,Households,AvgExpenditure,Total
CustomerNeighborhood,,,
Neighborhood_01,"5,000","1,500","7,500,000"
Neighborhood_02,"4,500","1,400","6,300,000"
Neighborhood_03,"7,500","1,250","9,375,000"
Neighborhood_04,"11,000","1,000","11,000,000"
Neighborhood_05,"2,500","1,400","3,500,000"


In [12]:
# Loop over each column to normalize its values
# Add a new column 'Attractiveness' which is the sum of all normalized values row-wise
def store_attractiveness(dataframe):
    dataframe_new = dataframe.copy().astype(float)
    rows, columns = dataframe_new.shape
    for j in range(columns):
        column_min = dataframe_new.iloc[:,j].min()
        column_max =  dataframe_new.iloc[:,j].max()
        for i in range(rows):
            numerator = dataframe_new.iloc[i,j] - column_min
            denominator = column_max - column_min
            dataframe_new.iloc[i,j] = (numerator / denominator)
    dataframe_new['Attractiveness'] = dataframe_new.sum(axis=1)
    return dataframe_new 
    
df_stores_scaled = store_attractiveness(df_stores)
df_stores_scaled

,Size,Parking spaces,BusLanes,Accessibility,Location,Attractiveness
StoreLocation,,,,,,
Shopping_Mall_01,1.0,0.75,0.50,1.0,0.0,3.25
Shopping_Mall_02,1.0,0.50,0.00,0.5,0.0,2.00
Waterfront_Park,0.0,1.00,0.25,0.0,1.0,2.25
Church_Street,0.5,0.00,1.00,0.0,0.5,2.00


In [13]:
# Extract the store names as a list from the index of df_stores_scaled
store_names = df_stores_scaled.index.values.tolist()

# Extract the neighborhood names as a list from the index of df_distance
neighborhood_names = df_distance.index.values.tolist()

# Create pairs of neighborhood-store combinations using list comprehension
pairs = [(x,y) for x in neighborhood_names for y in store_names]

In [14]:
#Demand probability
decay_exponent = 2 

# Calculate the attractiveness-distance ratio for each neighborhood-store pair
attract_by_dist_ratio = {}
for pair in pairs:
    attract_by_dist_ratio[pair] = df_stores_scaled.loc[pair[1],'Attractiveness'] / \
                                df_distance.loc[pair[0],pair[1]]**decay_exponent

# Calculate the demand probability for each neighborhood-store pair
demand_probability = {}
for pair in pairs:
    demand_probability[pair] = attract_by_dist_ratio[pair] / \
                            sum([v for k,v in attract_by_dist_ratio.items() if k[0] == pair[0]])

In [15]:
#Expected_sales_per_pair
expected_sales_per_pair = {}
for pair in pairs:
    expected_sales_per_pair[pair]= demand_probability[pair] * df_census.loc[pair[0],'Total']

In [16]:
#Expected_sales_per_store
expected_sales_per_store = {}
for store in store_names:
    expected_sales_per_store[store] = sum([v for k,v in expected_sales_per_pair.items() if k[1]==store])

In [17]:
# Convert the dictionary of expected sales per store into a DataFrame
expected_sales = pd.DataFrame.from_dict(expected_sales_per_store, orient='index').rename(
                columns={0: 'Expected_Sales'})['Expected_Sales'].apply(
                lambda x: '${:,.2f}'.format(x)).to_frame()

expected_sales

,Expected_Sales
Shopping_Mall_01,"$17,702,778.80"
Shopping_Mall_02,"$11,022,437.97"
Waterfront_Park,"$1,872,641.43"
Church_Street,"$7,077,141.81"
